# Abgleich von Artikel zwischen Wikisource und Wikidata

Im Rahmen des de.wikisource-Projekts zum [Album der sächsischen Industrie](https://de.wikisource.org/wiki/Album_der_S%C3%A4chsischen_Industrie) werden die Metadaten der einzelnen Aritkel in Wikidata angelegt.

Dieses Skript dient zur Analyse und Vergleich zwischen dem Wikidatabestand und den de.wikisource-Seiten

## 1. Abfrage der Seiten auf de.wikisource
Alle Einträge einer bestimmten Kategorie werden via Mediawiki-API abgerufen. Für jede wikisource-Seite wird überprüft, ob das wikidata-Item vorhanden ist.

Bei fehlendem Wikidata-Item wird die Seite aufgerufen und die Vorlage ```{{Textdaten}}``` ausgelesen und für den Wikidata-Import notwendige Metadaten ausgelesen und in Array geschrieben:

* P1476 (Titel)
* P577 (Erscheinungsdatum)
* P1433 (Erschienen in)
* P304 (Seite)
* P433 (Heft Nr)
* P921 (Schlagwort)
* P50 (Autor - wenn QID für AutorInnen vorhanden)
* P2093 (Autoren - Textstring)

Ergänze Author und Main Subject aus der Textdaten-Vorlage: Beispiel https://de.wikisource.org/wiki/Der_Magenkrampf


## 2. Aufbau einer TSV-Liste für den QuickStatement-Import
Basierend auf den zuvor erzeugten Ergebnis-Array ```resultSet``` wird ein [QuickStatements-Import](https://www.wikidata.org/wiki/Help:QuickStatements) aufgebaut ([Ergebnis](#qs)).

In [5]:
import requests
import re
import json
import urllib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import time

endpoint_url = "https://query.wikidata.org/sparql"

baseOutputFile = 'output/gartenlaube'+str(time.time())[0:10]+'.json'
exitQIDFile = 'output/existQID'+str(time.time())[0:10]+'.txt'
QSFile = 'output/qs'+str(time.time())[0:10]+'.tsv'
try:
    os.remove(exitQIDFile)
    os.remove(QSFile)
except:
    pass

S = requests.Session()
artikelCounter = 1
resultSet = []

def getWikibaseIDviaMwapi(pagetitle,URL):
    param_wbase = { 'action' : 'query', 'prop' : 'pageprops', 'ppprop': 'wikibase_item',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    try:
        for page in DATA["query"]["pages"]:
            #print(page)
            try:
                return DATA["query"]["pages"][page]["pageprops"]["wikibase_item"]
            except KeyError:
                pass
    except KeyError:
        pass

    
def getWikibaseIDviaSPARQL(pagetitle):
    query = """SELECT * WHERE {
      ?gartenlaubeArtikel wdt:P1433 wd:Q655617.
      ?article schema:about ?gartenlaubeArtikel.
      ?article schema:isPartOf <https://de.wikisource.org/>.
      ?article schema:name ?pageTitle.
      FILTER(STR(?pageTitle)='"""+str(pagetitle)+"""')
    }"""
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()
    #results = get_results(endpoint_url, query)
    #for result in results["results"]["bindings"]:
        #print(result["gartenlaubeArtikel"]["value"])

    
def getOldIDviaMwapi(pagetitle):
    param_wbase = { 'action' : 'query', 'prop' : 'info',
             'redirects': 1, 'titles': pagetitle,'format': "json" }
    R = S.get(url=URL, params=param_wbase)
    DATA = R.json()
    for page in DATA["query"]["pages"]:
        #print(page)
        try:
            return DATA["query"]["pages"][page]["lastrevid"]
        except KeyError:
            pass   

def parseWikiAPI(URL,PARAMS):
    R = S.get(url=URL, params=PARAMS)
    DATA = R.json()
    return DATA


#Run through Category
def runthroughCat(DATA,count=0):
    try:
        while DATA['query']:
            #print(str(artikelCounter))
            #print(resultSet)
            global artikelCounter
            artikelCounter = ParseArtikelMetadata(DATA,artikelCounter)

            try:
                #print(DATA['continue']['cmcontinue'])
                PARAMS['cmcontinue'] = DATA['continue']['cmcontinue']
                DATA = parseWikiAPI(URL,PARAMS)
            except KeyError:
                break
    except KeyError:
        pass


#Parse via Subcategories
def runthroughSubCat(kategorie):
    param_SubCat = PARAMS
    param_SubCat["cmtype"]="subcat"
    param_SubCat["cmtitle"] = kategorie
    DATACAT = parseWikiAPI(URL,param_SubCat)
    #print(DATACAT["query"])
    try:
        #print(str(artikelCounter))
        #print(resultSet)
        for subcat in DATACAT["query"]["categorymembers"]:
            print (">>=============================================")
            print (subcat["title"])
            PARAMS["cmtitle"] = subcat["title"]
            PARAMS.pop('cmtype',None)
            #print(PARAMS)
            DATA = parseWikiAPI(URL,PARAMS)
            runthroughCat(DATA)
    except KeyError:
        pass
    

def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url,agent="Wikisource:Die_Gartenlaube")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


URL = "https://de.wikisource.org/w/api.php"
#?action=query&list=categorymembers&cmlimit=500&cmtitle=Kategorie:Album_der_S%C3%A4chsischen_Industrie&format=json
PARAMS = {
    'action': "query",
    'list': "categorymembers",
    'cmtitle': "Kategorie:Die_Gartenlaube_(1853)_Artikel",
    'cmlimit': 500,
    'format': "json"
}

def writeQuickStatements(resultSet):
    print(resultSet)
    f = open(QSFile,'a')
    for result in resultSet:
        try:
            reference = '\tS143\tQ15522295\tS4656\t"'+result["permLink"]+'"'
        except:
            reference = ''

        
        
        print ("CREATE",file=f)
        #Labels, Descriptions
        print ('LAST\tSdewikisource\t"'+result["Sdewikisource"]+'"',file=f)
        print ('LAST\tLde\t"'+result["titel"]+'"',file=f)
        print ('LAST\tLen\t"'+result["titel"]+'"',file=f)
        print ('LAST\tLnl\t"'+result["titel"]+'"',file=f)
        try:
            print ('LAST\tDen\t"'+result["enDesc"]+'"',file=f)
        except KeyError:
            pass
        try:
            print ('LAST\tDde\t"'+result["deDesc"]+'"',file=f)
        except KeyError:
            pass
        try:
            print ('LAST\tDnl\t"'+result["nlDesc"]+'"',file=f)
        except KeyError:
            pass
        
        #Instance of Article
        print ('LAST\tP31\tQ191067',file=f)
        
        if(len(result["autorQID"])>0):
            try:
                for autcunt, autoren in enumerate(result["autorQID"]):
                    try:
                        print ('LAST\tP50\t'+autoren+
                               '\tP1932\t"'+result["autor"][autcunt]+'"'+
                               '\tP1545\t"'+str(autcunt+1)+'"'+reference,file=f)
                    except TypeError:
                        for autorenStr in result["autorStr"]:
                            if(autorenStr=='unbekannt'):
                                print ('LAST\tP2093\tsomevalue'+reference,file=f)
                            elif(autorenStr!=""):
                                print ('LAST\tP2093\t"'+autorenStr+'"'+reference,file=f)
            except KeyError:
                try:
                    for autorenStr in result["autorStr"]:
                        if(autorenStr=='unbekannt'):
                            print ('LAST\tP2093\tsomevalue'+reference)                    
                        elif(autorenStr!=""):
                            print ('LAST\tP2093\t"'+autorenStr+'"'+reference,file=f)
                except KeyError:
                    pass
        else:
            try:
                for autorenStr in result["autorStr"]:
                    if(autorenStr=='unbekannt'):
                        print ('LAST\tP2093\tsomevalue'+reference,file=f)                
                    elif(autorenStr!=""):
                        print ('LAST\tP2093\t"'+autorenStr+'"'+reference,file=f)
            except KeyError:
                pass 

        print ('LAST\tP1476\tde:"'+result["titel"]+'"'+reference,file=f)
        if(re.search('^Briefkasten',result["titel"])):
                print ('LAST\tP31\tQ651270'+reference,file=f)
        if(re.search('^Berichtigung',result["titel"])):
                print ('LAST\tP31\tQ1348305'+reference,file=f)

        try:
            if(result["untertitel"]!=""):
                print ('LAST\tP1680\tde:"'+result["untertitel"]+'"'+reference,file=f)
        except KeyError:
            pass
        print ('LAST\tP407\tQ188',file=f)
        print ('LAST\tP577\t+'+result["erscheinungsjahr"].strip()+'-00-00T00:00:00Z/9'+reference,file=f)
        try:
            print ('LAST\tP304\t"'+result["seite"]+'"'+reference,file=f)
            seiteasQual = '\tP304\t"'+result["seite"]+'"'
        except KeyError:
            seiteasQual =""
            pass
        try:
            print ('LAST\tP433\t"'+result["heft"]+'"'+reference,file=f)
            heftasQual = '\tP433\t"'+result["heft"]+'"'
        except KeyError:
            heftasQual = ""
            pass

        print ('LAST\tP1433\t'+result["gartenlaube"]+
               seiteasQual+
               heftasQual+
               '\tP577\t+'+result["erscheinungsjahr"].strip()+'-00-00T00:00:00Z/9'+reference,file=f)
        try:
            print('LAST\tP921\t'+result["mainSubjectQID"]+reference,file=f)
        except (KeyError, TypeError):
            pass
        
query = """SELECT * WHERE {
  ?gartenlaubeArtikel wdt:P1433 wd:Q655617.
  ?wsPage schema:about ?gartenlaubeArtikel;
    schema:isPartOf <https://de.wikisource.org/>.
  ?wsPage schema:name ?wsPageName
}"""

allGartenlaubeArtikel = get_results(endpoint_url, query)

def ParseArtikelMetadata(DATA,artikelCounter):
    #Variablen artikelCounter und resultset entweder hier "lokal" deklarieren oder global außerhalb vor Fkt-Call
    #artikelCounter = 1
    #resultSet = []
    #print(artikelCounter)
    resultDet = {}
    resultExistQID = []
    #print(DATA)
    
    #Fetch all existing Gartenlaube-Artikel in Wikidata and use the Dict to check for existing WD-Items (unless the wikisource-API hasn't every Wikibase-ID stored...)

    
    for artikel in DATA["query"]["categorymembers"]:
        wikisourceTitel = artikel["title"]+ " https://de.wikisource.org/?curid="+str(artikel["pageid"])

        #Get Wikidata QID
        #?action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles=Die_Leinen-Damastfabrik_und_Bleiche_von_J._G._Lieske_%26_H%C3%A4bler_in_Gro%C3%9F-Sch%C3%B6nau
        resultDet = {}

        #wdid = getWikibaseIDviaMwapi(artikel["title"],URL)
        
        
        wdid = None
        for result in allGartenlaubeArtikel["results"]["bindings"]:
            if(result["wsPageName"]["value"]==artikel["title"]):
                wdid = re.sub("http://www.wikidata.org/entity/","",result["gartenlaubeArtikel"]["value"])
        if(wdid == None):
            wdid = getWikibaseIDviaMwapi(artikel["title"],URL)
            
        print(wdid)
        
        if(wdid==None):
            print("#"+str(artikelCounter)+" we need a wikidata item for "+wikisourceTitel)
            param_text = { 'action' : 'query', 'prop' : 'revisions', 'rvprop': 'content',
             'rvlimit': 1, 'titles': artikel["title"],'format': "json" }
            #action=query&prop=revisions&rvlimit=1&rvprop=content&format=json
            #&titles=Die%20Glas-%20und%20Chemische%20Fabrik%20von%20Fr.%20Chr.%20Fikentscher%20bei%20Zwickau
            R = S.get(url=URL, params=param_text)
            textdata = R.json()
            #print(DATA)
            resultDet["Sdewikisource"] = artikel["title"]

            oldid = getOldIDviaMwapi(artikel["title"])
            permLink = "https://de.wikisource.org/w/index.php?title="+urllib.parse.quote_plus(artikel["title"])+"&oldid="+str(oldid)
            print(permLink)
            resultDet["permLink"] = permLink

            for page in textdata["query"]["pages"]:
                test_str = textdata["query"]["pages"][page]["revisions"][0]["*"]
                #print(test_str)
                #regex = r"\|TITEL=(.*)"
                regex = r"\|TITEL=((\[\[((.*(?<=\|)(.*?))|(.*?))\]\])|(.*))"
                matches = re.finditer(regex, test_str, re.MULTILINE)
                for matchNum, match in enumerate(matches, start=1):
                    if(match.group(5)!=None):
                        print (match.group(5))
                        resultDet["titel"] = match.group(5).strip()
                    elif(match.group(7)!=None):
                        print (match.group(7))
                        resultDet["titel"] = match.group(7).strip()
                    else:
                        print(match.group(3))
                        resultDet["titel"] = match.group(3).strip()
                        

                '''for matchNum, match in enumerate(matches, start=1):
                    print("Titel: "+ match.group(1))
                    titel = match.group(1)
                    resultDet["titel"] = titel'''

                regex = r"\|SUBTITEL=(.*)"
                matches = re.finditer(regex, test_str, re.MULTILINE)

                for matchNum, match in enumerate(matches, start=1):
                    print("Untertitel: "+ match.group(1))
                    untertitel = match.group(1)
                    resultDet["untertitel"] = untertitel

                regex = r"\|JAHR=(.*)"
                matches = re.finditer(regex, test_str, re.MULTILINE)

                for matchNum, match in enumerate(matches, start=1):
                    print("Erscheiungsjahr: "+ match.group(1))   
                    erscheinungsjahr = match.group(1)
                    resultDet["erscheinungsjahr"] = erscheinungsjahr

                regex = r"\|Seite=(.*)"
                matches = re.finditer(regex, test_str, re.MULTILINE)

                for matchNum, match in enumerate(matches, start=1):
                    print("Seite: "+ match.group(1))   
                    seite = match.group(1)
                    if seite != "":
                        resultDet["seite"] = seite

                regex = r"\|Heft=(.*)"
                matches = re.finditer(regex, test_str, re.MULTILINE)

                for matchNum, match in enumerate(matches, start=1):
                    print("Heft: "+ match.group(1))   
                    heft = match.group(1)
                    
                    regex = r"\[\[(.*)\|(.*)\]\]"
                    matches = re.finditer(regex, heft, re.MULTILINE)
                    for matchNum, match in enumerate(matches, start=1):
                        heft = re.sub(regex,match.group(2),heft)
                    
                    resultDet["heft"] = heft    

                regex = r"\|AUTOR=(.*)"
                regexAut = r"\[\[((.*(?<=\|)(.*?))|(.*?))\]]"
                matches = re.finditer(regex, test_str, re.MULTILINE)
                for matchNum, match in enumerate(matches, start=1):
                    print("Autor Gesamt: "+ match.group(1))   
                    autor = match.group(1).strip()
                    matches = re.finditer(regexAut, autor, re.MULTILINE)
                    autorLis = []
                    autorQIDLis = []
                    autorStr = []
                    for matchNumAut, matchAut in enumerate(matches,start=1):
                        try:
                            print("Autor: "+ matchAut.group(3))
                            autorLis.append(matchAut.group(3))
                            autorStr.append(matchAut.group(3).strip())
                        except:
                            print("Autor: "+ matchAut.group(1))
                            autorLis.append(matchAut.group(1))
                            autorStr.append(matchAut.group(1).strip())
                        #print("Autor: "+ matchAut.group(1))    
                        autorqid = getWikibaseIDviaMwapi(matchAut.group(1),URL)
                        print("autorQID: "+str(autorqid))
                        autorQIDLis.append(autorqid)

                    resultDet["autor"] = autorLis    
                    print (resultDet["autor"])
                    resultDet["autorQID"] = autorQIDLis
                    print (resultDet["autorQID"])
                    if(len(autorStr) ==0):
                        resultDet["autorStr"] = [autor]
                    else:
                        resultDet["autorStr"] = autorStr

                regex = r"\|WIKIPEDIA=(.*)"
                matches = re.finditer(regex, test_str, re.MULTILINE)
                for matchNum, match in enumerate(matches, start=1):
                    print("Main Subject: "+ match.group(1))   
                    mainsub = match.group(1)
                    resultDet["mainSubject"] = mainsub
                    msQID = getWikibaseIDviaMwapi(mainsub,"https://de.wikipedia.org/w/api.php")
                    resultDet["mainSubjectQID"] = msQID             
                    print("mainSubQID: "+str(msQID))


                #Fixe Parameter
                resultDet["gartenlaube"] = "Q655617"
                print(resultDet["gartenlaube"])
                deDesc = ""
                enDesc = ""
                nlDesc = ""
                try:
                    deDesc = "Artikel in: Die Gartenlaube, "+erscheinungsjahr + ", Heft "+heft
                    resultDet["deDesc"] = deDesc
                    print(deDesc)
                    
                    enDesc = "german article in Die Gartenlaube"+", "+erscheinungsjahr+ ", no. "+heft
                    resultDet["enDesc"] = enDesc
                    print(enDesc)
                
                    nlDesc = "redactioneel artikel in Die Gartenlaube ("+erscheinungsjahr+"), nr. "+heft
                    resultDet["nlDesc"] = nlDesc
                    print(nlDesc)
                except NameError:
                    pass

                try:
                    test = resultDet["titel"]
                    resultSet.append(dict(resultDet))
                   
                except KeyError:
                    pass


            writeQuickStatements([resultDet])
        else:
            print("#"+str(artikelCounter)+" "+wdid+" already exists for "+wikisourceTitel)
            #Hier koennten die vorhanenen QIDs gesammelt werden und fuere eine SPARQL-Query mit großem VALUE []
            #genutzt werden um die Datenqualität der vorhandenen Einträge zu überprüfen
            #Elemente ohne Titel, P31
            #Elemente ohne Heft, Issue, Published in, Publication Date
            resultExistQID.append(wdid)
            fa = open(exitQIDFile,'a')
            print(wdid,file=fa)
            

        artikelCounter = artikelCounter+1
        print("--------------------------------------------------------------------------")
    return artikelCounter
    
DATA = parseWikiAPI(URL,PARAMS)
#print(DATA)
count = 0

R = S.get(url=URL, params=PARAMS)
DATA = R.json()


runthroughSubCat("Kategorie:Die_Gartenlaube_Artikel")
try:
    os.remove(baseOutputFile)
except:
    pass
%store json.dumps(resultSet) >> $baseOutputFile


>>=============================================
Kategorie:Die Gartenlaube (1853) Artikel
Q61715997
#1 Q61715997 already exists for Abdul Medjid https://de.wikisource.org/?curid=342602
--------------------------------------------------------------------------
Q19125040
#2 Q19125040 already exists for Abschreckungstheorie in Schnupftaback https://de.wikisource.org/?curid=338866
--------------------------------------------------------------------------
Q19125952
#3 Q19125952 already exists for Alexander von Humboldt (Die Gartenlaube 1853) https://de.wikisource.org/?curid=288567
--------------------------------------------------------------------------
Q19126927
#4 Q19126927 already exists for Alles schon dagewesen https://de.wikisource.org/?curid=337957
--------------------------------------------------------------------------
Q19127340
#5 Q19127340 already exists for Alte Schulzucht https://de.wikisource.org/?curid=340640
------------------------------------------------------------------

--------------------------------------------------------------------------
Q19188262
#75 Q19188262 already exists for Ein Anfang https://de.wikisource.org/?curid=290632
--------------------------------------------------------------------------
Q19188265
#76 Q19188265 already exists for Ein Antwortschreiben https://de.wikisource.org/?curid=336725
--------------------------------------------------------------------------
Q19190270
#77 Q19190270 already exists for Ein armer Teufel https://de.wikisource.org/?curid=337346
--------------------------------------------------------------------------
Q19188284
#78 Q19188284 already exists for Ein Asyl für die verwahrloste Jugend https://de.wikisource.org/?curid=193229
--------------------------------------------------------------------------
Q19188354
#79 Q19188354 already exists for Ein Berliner Gaunerwitz https://de.wikisource.org/?curid=342621
--------------------------------------------------------------------------
Q19188483
#80 Q19188483 a

Q19176175
#154 Q19176175 already exists for Die Heilung einer Wahnsinnigen https://de.wikisource.org/?curid=342607
--------------------------------------------------------------------------
Q19153146
#155 Q19153146 already exists for Das Heimweh (Die Gartenlaube 1853) https://de.wikisource.org/?curid=340413
--------------------------------------------------------------------------
Q19148216
#156 Q19148216 already exists for Das Herrmannsdenkmal https://de.wikisource.org/?curid=300907
--------------------------------------------------------------------------
Q19211265
#157 Q19211265 already exists for Herz und Blutlauf https://de.wikisource.org/?curid=290382
--------------------------------------------------------------------------
Q19211385
#158 Q19211385 already exists for Hexen in Nordamerika https://de.wikisource.org/?curid=310724
--------------------------------------------------------------------------
Q62610315
#159 Q62610315 already exists for Historische Blumenlese (1853/8) htt

--------------------------------------------------------------------------
Q19220484
#231 Q19220484 already exists for Musikalische Stenographie https://de.wikisource.org/?curid=340953
--------------------------------------------------------------------------
Q19189349
#232 Q19189349 already exists for Ein Mutterherz https://de.wikisource.org/?curid=332060
--------------------------------------------------------------------------
Q19178564
#233 Q19178564 already exists for Die Näherin https://de.wikisource.org/?curid=338630
--------------------------------------------------------------------------
Q19178569
#234 Q19178569 already exists for Die Nähmaschine https://de.wikisource.org/?curid=181660
--------------------------------------------------------------------------
Q19221359
#235 Q19221359 already exists for Natürliche Brücke bei Kilkee https://de.wikisource.org/?curid=290231
--------------------------------------------------------------------------
Q19163549
#236 Q19163549 already

--------------------------------------------------------------------------
Q19236357
#311 Q19236357 already exists for Vor der Sündfluth – Im Krystallpalaste https://de.wikisource.org/?curid=247942
--------------------------------------------------------------------------
Q19237133
#312 Q19237133 already exists for Was das Leben und die Gesundheit des Menschen erhält? https://de.wikisource.org/?curid=340365
--------------------------------------------------------------------------
Q19237209
#313 Q19237209 already exists for Was in Amerika ein Schneider werden kann https://de.wikisource.org/?curid=289149
--------------------------------------------------------------------------
Q19153276
#314 Q19153276 already exists for Das Wasser https://de.wikisource.org/?curid=305558
--------------------------------------------------------------------------
Q19237366
#315 Q19237366 already exists for Wasser thuts freilich https://de.wikisource.org/?curid=336765
--------------------------------------

...

--------------------------------------------------------------------------
Q19209766
#12663 Q19209766 already exists for Gute Wintersuppe für den sparsamen Haushalt https://de.wikisource.org/?curid=297506
--------------------------------------------------------------------------
Q19212094
#12664 Q19212094 already exists for Hängekreuz mit Emailmosaik https://de.wikisource.org/?curid=388156
--------------------------------------------------------------------------
Q19212096
#12665 Q19212096 already exists for Hängerkleidchen mit Stickerei https://de.wikisource.org/?curid=388106
--------------------------------------------------------------------------
Q62617551
#12666 Q62617551 already exists for Heinrich Kiepert (Die Gartenlaube 1898/16) https://de.wikisource.org/?curid=506652
--------------------------------------------------------------------------
Q19210767
#12667 Q19210767 already exists for Helle Herrenkrawatten zu reinigen https://de.wikisource.org/?curid=388143
-----------------

Q19229455
#12721 Q19229455 already exists for Stickereistreifen zur Verzierung von Kleidern, Schürzen etc. https://de.wikisource.org/?curid=388084
--------------------------------------------------------------------------
Q19154077
#12722 Q19154077 already exists for Das „Tänzelfest“ in Kaufbeuren https://de.wikisource.org/?curid=391176
--------------------------------------------------------------------------
Q19230533
#12723 Q19230533 already exists for Tischläufer aus gekrepptem Seidenpapier https://de.wikisource.org/?curid=388046
--------------------------------------------------------------------------
Q19231350
#12724 Q19231350 already exists for Ueber den Einfluß der Kälte auf die Keimfähigkeit der Pflanzen https://de.wikisource.org/?curid=388137
--------------------------------------------------------------------------
Q19231753
#12725 Q19231753 already exists for Um abgeschnittene Blüten https://de.wikisource.org/?curid=388159
--------------------------------------------------

--------------------------------------------------------------------------
Q19193195
#12801 Q19193195 already exists for Einwickeltuch für Schuhe https://de.wikisource.org/?curid=388333
--------------------------------------------------------------------------
Q76557545
#12802 Q76557545 already exists for Elche https://de.wikisource.org/?curid=520855
--------------------------------------------------------------------------
Q62617667
#12803 Q62617667 already exists for Elen- und Säbel-Antilopen mit Jungen https://de.wikisource.org/?curid=474647
--------------------------------------------------------------------------
Q62617668
#12804 Q62617668 already exists for Engerlinge im Garten https://de.wikisource.org/?curid=486931
--------------------------------------------------------------------------
Q19193828
#12805 Q19193828 already exists for Erdbeerenschifferl https://de.wikisource.org/?curid=388348
--------------------------------------------------------------------------
Q66667506
#1

Q19216037
#12849 Q19216037 already exists for Knopfbefestigung an Paletots u. s. w. https://de.wikisource.org/?curid=388294
--------------------------------------------------------------------------
Q19216073
#12850 Q19216073 already exists for Kokosmatten zu reinigen https://de.wikisource.org/?curid=388300
--------------------------------------------------------------------------
Q19216244
#12851 Q19216244 already exists for Kopfputze zu Maskenkostümen https://de.wikisource.org/?curid=388252
--------------------------------------------------------------------------
Q62617710
#12852 Q62617710 already exists for Der Krieg in Südafrika (Die Gartenlaube 1899/25) https://de.wikisource.org/?curid=455363
--------------------------------------------------------------------------
Q62617714
#12853 Q62617714 already exists for Der Krieg in Südafrika (Die Gartenlaube 1899/27) https://de.wikisource.org/?curid=506249
--------------------------------------------------------------------------
Q191701

--------------------------------------------------------------------------
Q62617741
#12912 Q62617741 already exists for Vermehrung der Pflanzen im August und September https://de.wikisource.org/?curid=486932
--------------------------------------------------------------------------
Q19233157
#12913 Q19233157 already exists for Vermißten-Liste (Die Gartenlaube 1899) https://de.wikisource.org/?curid=373520
--------------------------------------------------------------------------
Q62617745
#12914 Q62617745 already exists for Verraten https://de.wikisource.org/?curid=388339
--------------------------------------------------------------------------
Q19235058
#12915 Q19235058 already exists for Verwendung von Aluminiumplatten in der Lithographie https://de.wikisource.org/?curid=361654
--------------------------------------------------------------------------
Q19239416
#12916 Q19239416 already exists for Wäsche wattierter Steppdecken https://de.wikisource.org/?curid=388327
-----------------